In [59]:
import numpy as np
def KMeansPlusPlus(data, k):
    n = data.shape[0]
    
    #Initialize the first centroid
    centroids = data[np.random.choice(range(n),1),:]
    
    #broadcast the data sets to calculate the distance conveniently
    data_expand = data[:, np.newaxis, :]
    
    while centroids.shape[0] < k :
        
        #Calculate the distance between 
        pre_distance = (data_expand - centroids) ** 2
        distance = np.sum(pre_distance, axis=2)
        
        #mark the location of minimum distance for every centroid
        min_dist = np.zeros(distance.shape)
        min_dist[range(distance.shape[0]), np.argmin(distance, axis=1)] = 1
        
        #Calculate the normal constant of the distribution
        normal_cons = np.sum(distance[min_dist == 1])
        
        #Get the distribution for sampling new centroids
        distribution = np.min(distance, axis=1)/normal_cons
        
        #Obtain the next centroid and append
        centroids = np.r_[centroids, data[np.random.choice(range(n),1,p=distribution), :]]
    
    return centroids

In [55]:
k = 3
# if all of inputs is correct, it works well.
data1 = np.random.randn(1000,2)

def random_pick(distribution,l):
    cutoffs = np.cumsum(distribution)
    idx = cutoffs.searchsorted(np.random.uniform(0, cutoffs[-1],l))
    return idx

823

In [58]:
def random_pick(distribution,l):
    cutoffs = np.cumsum(distribution)
    idx = cutoffs.searchsorted(np.random.uniform(0, cutoffs[-1],l))
    return idx

In [60]:
def ScalableKMeansPlusPlus(data, k, l):
    
    centroids = data[np.random.choice(range(data.shape[0]),1), :]
    data_expand = data[:, np.newaxis, :]
    
    #Calculate the iteration times
    pre_distance = (data_expand - centroids) ** 2
    distance = np.sum(pre_distance, axis=2)
    min_dist = np.zeros(distance.shape)
    min_dist[range(distance.shape[0]), np.argmin(distance, axis=1)] = 1
    normal_cons = np.sum(distance[min_dist == 1])
    iter = int(np.log(normal_cons))
    
    for i in range(iter):
        pre_distance = (data_expand - centroids) ** 2
        distance = np.sum(pre_distance, axis=2)
        
        #mark the location of minimum distance for every centroid
        min_dist = np.zeros(distance.shape)
        min_dist[range(distance.shape[0]), np.argmin(distance, axis=1)] = 1
        
        #Calculate the normal constant of the distribution
        normal_cons = np.sum(distance[min_dist == 1])
        
        #Get the distribution for sampling new centroids
        distribution = np.min(distance, axis=1)/normal_cons
        
        center_index = random_pick(distribution,l)
        center = X[center_index,]
        
        centroids = np.vstack((centroids, center))
        # Finally we get a set of centroids which is higher than k

    ## reduce k*l to k using KMeans++
    pre_distance = (data_expand - centroids) ** 2
    distance = np.sum(pre_distance, axis=2)
    min_dist = np.zeros(distance.shape)
    min_dist[range(distance.shape[0]), np.argmin(distance, axis=1)] = 1
    weights = np.array([np.count_nonzero(min_dist[:, i]) for i in range(centroids.shape[0])]).reshape(-1,1)
    prob_distribution = weights/np.sum(weights)
    centroids = data[np.random.choice(range(weights.shape[0]),k,p=prob_distribution.ravel()),:]
    
    return centroids

In [61]:
k = 3
l = 2
data1 = np.random.randn(1000,2)
ScalableKMeansPlusPlus(data1,k,l)

array([[ 0.62064658,  0.72589247],
       [ 0.62064658,  0.72589247],
       [-0.52097528,  0.10060321]])

In [4]:
X.shape[0]

1000

In [7]:
## paition the data set X
from pyspark import SparkContext
sc = SparkContext(master = 'local[*]')

In [1]:
## Simulate data
k = 20
n = 10000
d = 15

## simulate k centers from 15-dimensional spherical Gaussian distribution 
mean = np.hstack(np.zeros((d,1)))
cov = np.diag(np.array([1,10,100]*5))
centers = np.random.multivariate_normal(mean, cov, k)

## Simulate n data
for i in range(k):
    mean = centers[i]
    if i == 0:
        data = np.random.multivariate_normal(mean, np.diag(np.ones(d)) , int(n/k+n%k))
    else:
        data = np.append(data, np.random.multivariate_normal(mean, np.diag(np.ones(d)) , int(n/k)), axis = 0) 

In [ ]:
X=data

In [11]:
from pyspark.mllib.clustering import KMeans, KMeansModel
from numpy import array
from math import sqrt

# Load and parse the data

data = sc.parallelize(X)

# Build the model (cluster the data)
clusters = KMeans.train(data, 2, maxIterations=10,
        runs=10, initializationMode="kmeans||")

# Evaluate clustering by computing Within Set Sum of Squared Errors
def error(point):
    center = clusters.centers[clusters.predict(point)]
    return sqrt(sum([x**2 for x in (point - center)]))

WSSSE = data.map(lambda point: error(point)).reduce(lambda x, y: x + y)
print("Within Set Sum of Squared Error = " + str(WSSSE))

/Users/fwen/anaconda/lib/python3.5/site-packages/pyspark/mllib/clustering.py:176: UserWarning: Support for runs is deprecated in 1.6.0. This param will have no effect in 1.7.0.
  "Support for runs is deprecated in 1.6.0. This param will have no effect in 1.7.0.")


Within Set Sum of Squared Error = 1024.1977821724317


In [18]:
clusters.centers

[array([ 0.77468133, -0.1934682 ]), array([-0.80748991,  0.2617322 ])]

In [19]:
clusters.clusterCenters

[array([ 0.77468133, -0.1934682 ]), array([-0.80748991,  0.2617322 ])]

In [27]:
n = [clusters.predict(x) for x in X] 

In [29]:
?KMeans.train